In [1]:
import json
import random
import gzip
import tiktoken # for token counting
import numpy as np
from collections import defaultdict

In [2]:
instruction = "Classify the input as either 'Likely fake.' or 'Not suspected to be fake.'."

In [3]:
def openai_format(news_single, label_single):
    return {"messages": [{"role": "system", "content":instruction}, {"role": "user", "content":news_single}, {"role": "assistant", "content":label_single}]}

In [4]:
def split_news(text_file):
    with open(text_file, "r") as f:
        string_whole = f.read()
    list_news = string_whole.split('#')
    return list_news

In [5]:
def dicts_to_jsonl(data_list: list, filename: str, compress: bool = False) -> None:
    # https://ml-gis-service.com/index.php/2022/04/27/toolbox-python-list-of-dicts-to-jsonl-json-lines/
    """
    Method saves list of dicts into jsonl file.
    :param data: (list) list of dicts to be stored,
    :param filename: (str) path to the output file. If suffix .jsonl is not given then methods appends
        .jsonl suffix into the file.
    :param compress: (bool) should file be compressed into a gzip archive?
    """
    sjsonl = '.jsonl'
    sgz = '.gz'
    # Check filename
    if not filename.endswith(sjsonl):
        filename = filename + sjsonl
    # Save data
    
    if compress:
        filename = filename + sgz
        with gzip.open(filename, 'w') as compressed:
            for ddict in data_list:
                jout = json.dumps(ddict) + '\n'
                jout = jout.encode('utf-8')
                compressed.write(jout)
    else:
        with open(filename, 'w') as out:
            for ddict in data_list:
                jout = json.dumps(ddict) + '\n'
                out.write(jout)

In [6]:
def news_to_dicts(news_file, label):
    news = split_news(news_file)
    list_dict = [openai_format(n, label) for n in news]
    return list_dict

In [7]:
list_dict = news_to_dicts('onion_article.txt', 'Likely fake.') + news_to_dicts('cnn_news.txt', 'Not suspected to be fake.')
random.shuffle(list_dict)

In [8]:
# split to training and validation dataset, then convert to jsonl
t = 51 
ft_train,  ft_valid = list_dict[:t], list_dict[t:]
dicts_to_jsonl(ft_train, "ft_train.jsonl")
dicts_to_jsonl(ft_valid, "ft_valid.jsonl")

## Data Checking

Modified from OpenAI sample scripts

In [9]:
data_path = "ft_train.jsonl"

In [10]:
# Load the dataset
with open(data_path, 'r', encoding='utf-8') as f:
    dataset = [json.loads(line) for line in f]

In [11]:
# Format error checks
format_errors = defaultdict(int)

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

No errors found


In [12]:
# token counting ultilities

encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")

In [13]:
# Warnings and tokens counts
n_missing_system = 0
n_missing_user = 0
n_messages = []
convo_lens = []
assistant_message_lens = []

for ex in dataset:
    messages = ex["messages"]
    if not any(message["role"] == "system" for message in messages):
        n_missing_system += 1
    if not any(message["role"] == "user" for message in messages):
        n_missing_user += 1
    n_messages.append(len(messages))
    convo_lens.append(num_tokens_from_messages(messages))
    assistant_message_lens.append(num_assistant_tokens_from_messages(messages))
    
print("Num examples missing system message:", n_missing_system)
print("Num examples missing user message:", n_missing_user)
print_distribution(n_messages, "num_messages_per_example")
print_distribution(convo_lens, "num_total_tokens_per_example")
print_distribution(assistant_message_lens, "num_assistant_tokens_per_example")
n_too_long = sum(l > 4096 for l in convo_lens)
print(f"\n{n_too_long} examples may be over the 4096 token limit, they will be truncated during fine-tuning")

Num examples missing system message: 0
Num examples missing user message: 0

#### Distribution of num_messages_per_example:
min / max: 3, 3
mean / median: 3.0, 3.0
p5 / p95: 3.0, 3.0

#### Distribution of num_total_tokens_per_example:
min / max: 90, 698
mean / median: 328.5686274509804, 317.0
p5 / p95: 262.0, 435.0

#### Distribution of num_assistant_tokens_per_example:
min / max: 4, 6
mean / median: 4.980392156862745, 4.0
p5 / p95: 4.0, 6.0

0 examples may be over the 4096 token limit, they will be truncated during fine-tuning


In [14]:
# Pricing and default n_epochs estimate
MAX_TOKENS_PER_EXAMPLE = 4096

TARGET_EPOCHS = 3
MIN_TARGET_EXAMPLES = 100
MAX_TARGET_EXAMPLES = 25000
MIN_DEFAULT_EPOCHS = 1
MAX_DEFAULT_EPOCHS = 25

n_epochs = TARGET_EPOCHS
n_train_examples = len(dataset)
if n_train_examples * TARGET_EPOCHS < MIN_TARGET_EXAMPLES:
    n_epochs = min(MAX_DEFAULT_EPOCHS, MIN_TARGET_EXAMPLES // n_train_examples)
elif n_train_examples * TARGET_EPOCHS > MAX_TARGET_EXAMPLES:
    n_epochs = max(MIN_DEFAULT_EPOCHS, MAX_TARGET_EXAMPLES // n_train_examples)

n_billing_tokens_in_dataset = sum(min(MAX_TOKENS_PER_EXAMPLE, length) for length in convo_lens)
print(f"Dataset has ~{n_billing_tokens_in_dataset} tokens that will be charged for during training")
print(f"By default, you'll train for {n_epochs} epochs on this dataset")
print(f"By default, you'll be charged for ~{n_epochs * n_billing_tokens_in_dataset} tokens")

Dataset has ~16757 tokens that will be charged for during training
By default, you'll train for 3 epochs on this dataset
By default, you'll be charged for ~50271 tokens


## Upload to OpenAI

In [15]:
import os
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [16]:
openai.File.create(
  file=open("ft_train.jsonl", "rb"),
  purpose='fine-tune'
)

<File file id=file-XYM0IZ0sA56oMxt3VKTOgVN5 at 0x10e58f360> JSON: {
  "object": "file",
  "id": "file-XYM0IZ0sA56oMxt3VKTOgVN5",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 85746,
  "created_at": 1697668301,
  "status": "uploaded",
  "status_details": null
}

In [17]:
openai.File.create(
  file=open("ft_valid.jsonl", "rb"),
  purpose='fine-tune'
)

<File file id=file-0U0Zec4vxYLMbiQHD1XkvxBT at 0x10f9198b0> JSON: {
  "object": "file",
  "id": "file-0U0Zec4vxYLMbiQHD1XkvxBT",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 29906,
  "created_at": 1697668302,
  "status": "uploaded",
  "status_details": null
}

## Start Fine-tuning

Obtain IDs of files from above file objects, then start fine-tuning.

In [18]:
openai.FineTuningJob.create(training_file="file-dlHUAfTQFHyJqxeXfGIUyevj", validation_file="file-juD22dDlnwtroUetln81e90I", model="gpt-3.5-turbo", hyperparameters={"n_epochs":3})

<FineTuningJob fine_tuning.job id=ftjob-BoIsGcBm2diBSGg174dcJ17Q at 0x10e58f090> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-BoIsGcBm2diBSGg174dcJ17Q",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1697668303,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-NGnL5tnUnu3nZEri5Tb21z0h",
  "result_files": [],
  "status": "validating_files",
  "validation_file": "file-juD22dDlnwtroUetln81e90I",
  "training_file": "file-dlHUAfTQFHyJqxeXfGIUyevj",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null,
  "error": null
}

Obtain fine-tuning job id from the above.

In [19]:
# Retrieve the state of a fine-tune
openai.FineTuningJob.retrieve("ftjob-BoIsGcBm2diBSGg174dcJ17Q")

<FineTuningJob fine_tuning.job id=ftjob-BoIsGcBm2diBSGg174dcJ17Q at 0x10d0c7d60> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-BoIsGcBm2diBSGg174dcJ17Q",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1697668303,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-NGnL5tnUnu3nZEri5Tb21z0h",
  "result_files": [],
  "status": "queued",
  "validation_file": "file-juD22dDlnwtroUetln81e90I",
  "training_file": "file-dlHUAfTQFHyJqxeXfGIUyevj",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null,
  "error": null
}

In [21]:
# List up to 10 events from a fine-tuning job
openai.FineTuningJob.list_events(id="ftjob-BoIsGcBm2diBSGg174dcJ17Q", limit=10)

<OpenAIObject list at 0x10d0e37c0> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-shdo1XgFdB7UD9BGXnj7jOcX",
      "created_at": 1697669546,
      "level": "info",
      "message": "The job has successfully completed",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-234yM31o2c8wkykKlJDqtkcU",
      "created_at": 1697669544,
      "level": "info",
      "message": "New fine-tuned model created: ft:gpt-3.5-turbo-0613:personal::8B9gl9uo",
      "data": {},
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-Hk5ndIFOBGGnqYYCLomwKrI0",
      "created_at": 1697669532,
      "level": "info",
      "message": "Step 151/153: training loss=0.00, validation loss=0.00",
      "data": {
        "step": 151,
        "train_loss": 7.152557373046875e-07,
        "valid_loss": 7.152557373046875e-07,
        "train_mean_token_acc

Obtain fine-tuned model number from the above.

The model training and validation test results can be obtained from the email sent by OpenAI after fine-tuning is completed.